## Demo: Stage2 processing for mooring data

**Stage 2 Overview**: This is the second processing stage that applies temporal corrections and deployment trimming to Stage1 NetCDF files. It focuses solely on time coordinate modifications and data trimming without changing the actual data values.

### What Stage2 Does:
- **Clock Offset Corrections**: Applies time corrections specified in YAML configuration files to fix instrument clock errors
- **Deployment Trimming**: Crops data to the actual deployment period (removes pre/post deployment data)  
- **Time Coordinate Only**: Only modifies the time coordinate - all data values remain unchanged
- **Metadata Updates**: Updates processing history and adds Stage2-specific attributes
- **File Renaming**: Converts `*_raw.nc` files to `*_use.nc` files indicating they're ready for analysis

### Input Files:
- Stage1 NetCDF files: `{mooring}_{serial}_raw.nc`
- Updated YAML configuration files with `clock_offset` values (determined from clock offset analysis)
- Deployment timing information in YAML files

### Output Files:
- Time-corrected NetCDF files: `{mooring}_{serial}_use.nc`
- Processing logs documenting applied corrections

### Processing Flow:
1. Load Stage1 (`*_raw.nc`) files
2. Read clock offset values from YAML configuration
3. Apply temporal corrections to time coordinate only
4. Trim data to deployment period (between deployment and recovery times)
5. Update metadata with processing history
6. Save as Stage2 (`*_use.nc`) files

### Clock Offset Workflow:
1. **Analyze**: Use `demo_clock_offset.ipynb` (which uses `oceanarray.clock_offset.py`) to determine timing corrections needed
2. **Update YAML**: Manually add `clock_offset` values (in seconds) to instrument configurations  
3. **Process Stage2**: Apply corrections and create `*_use.nc` files
4. **Verify**: Re-run clock offset analysis on `*_use.nc` files to confirm corrections

**Note**: Stage2 only modifies time coordinates and trims data - no quality control or data value modifications are performed. The `*_use.nc` files represent individual instruments with corrected timing and trimmed to deployment periods.

This notebook demonstrates the usage of the `oceanarray.stage2` module.

In [ ]:
import os

import yaml
import scipy.io
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
from datetime import datetime
import pandas as pd
from ctd_tools.writers import NetCdfWriter
from oceanarray import writers

moorlist = ['ds2_X_2012','ds2_X_2017','ds2_X_2018',
            'ds8_1_2012','ds9_1_2012','ds10_1_2012', 'ds11_1_2012','ds12_1_2012',
            'ds13_1_2012','ds14_1_2012','ds15_1_2012','ds16_1_2012','ds17_1_2012',
            'ds19_1_2012','ds18_1_2012','ds28_1_2017',
            'dsA_1_2018','dsB_1_2018','dsC_1_2018', 'dsD_1_2018','dsE_1_2018','dsF_1_2018',
            'dsM1_1_2017','dsM2_1_2017','dsM3_1_2017','dsM4_1_2017','dsM5_1_2017']

In [ ]:
from oceanarray.stage2 import Stage2Processor, process_multiple_moorings_stage2

# Simple usage
moorlist = ['dsE_1_2018']
basedir = '/Users/eddifying/Dropbox/data/ifmro_mixsed/ds_data_eleanor/'
results = process_multiple_moorings_stage2(moorlist, basedir)

In [ ]:
dir1 = '/Users/eddifying/Dropbox/data/ifmro_mixsed/ds_data_eleanor/moor/proc/dsE_1_2018/microcat'
fname = 'dsE_1_2018_7518_use.nc'
from pathlib import Path
data1 = xr.open_dataset(Path(dir1) / fname)

In [ ]:
data1

In [ ]:
data1.serial_number.values